Decorator <br>
예시) @staticmethod @classmethod <br>
<br>
장점
1. 중복 제거, 코드 간결, 공통함수 작성
2. 로깅, 프레임워크, 유효성 체크 -> 공통 기능으로 분리할 수 있다.
3. 조합해서 사용하기 편하다.

단점

1. 남발하면 가독성이 감소한다.
2. 특정 기능에 한정된 함수는 그냥 단일함수로 작성하는 게 나을 수 있다.
3. 디버깅 불편하다.




In [1]:
# 사용하기는 쉬은데 만들어 쓰는 것은 중급 이상의 실력을 요구한다.

import time


# enclosing scope start
def perf_clock(func):
    # Free variable, 매개변수도 포함됨

    def perf_clocked(*args):
        # 함수 시작 시간
        st = time.perf_counter()

        # 함수 실행

        result = func(*args)

        # 함수 종료 시간
        et = time.perf_counter() - st
        
        # 실행함수 이름
        name = func.__name__

        # 함수 매개변수
        arg_str = ', '.join(repr(arg) for arg in args)

        # 결과 출력
        print('[%0.5fs] %s(%s) -> %r' % (et, name, arg_str, result))
        return result
    
    return perf_clocked
# enclosing scope end

# 내부함수(perf_clocked)에서 외부함수(perf_clock)에서 넘어온 상태값(func)을
# 알고 있으니 일종의 closure이다.

Decorator VS Closure <br>

1. Closure가 Decorator 개념을 포함하고 있다.
2. Closure를 유지하면서 outer function은 매개변수로 target함수를 넣으면
 매개변수가 enclosing scope에 포함된다.
 그것이 nested function에서 사용되어야 한다. <br>

형태를 보자! <br>

def outer_function(target_function: object):

    free_variables.... (including target_function)


    def inner_function(parameters...):
        
        function_body...

        use the target_function.

        return ....

    
    return inner_function

In [2]:
def time_func(seconds):
    time.sleep(seconds)

def sum_func(*numbers):

    return sum(numbers)

In [3]:
# 데코레이터 미사용

none_deco1 = perf_clock(time_func)
print(none_deco1(1))
none_deco2 = perf_clock(sum_func)
print(none_deco2(*range(1,11)))
print('\n')

print(none_deco1, none_deco1.__code__.co_freevars)
print(none_deco2, none_deco2.__code__.co_freevars)

[1.01479s] time_func(1) -> None
None
[0.00000s] sum_func(1, 2, 3, 4, 5, 6, 7, 8, 9, 10) -> 55
55


<function perf_clock.<locals>.perf_clocked at 0x00000285B799AB80> ('func',)
<function perf_clock.<locals>.perf_clocked at 0x00000285B7C65040> ('func',)


In [4]:
# 데코레이터 사용

@perf_clock
def time_func(seconds):
    time.sleep(seconds)

@perf_clock
def sum_func(*numbers):
    """들어온 인수를 모두 합한다."""
    return sum(numbers)

time_func(1.3)
sum_func(*range(5))



[1.30788s] time_func(1.3) -> None
[0.00000s] sum_func(0, 1, 2, 3, 4) -> 10


10

decorator를 이해하는 데 필요한 것들 <br>

- Closure
- First-class function
- 가변인수 (*args, **kwargs)
- argument unpacking
- 파이썬이 소스코드를 불러오는 자세한 과정

# functools.wraps()

강의에 없는 것이나 Extending PyTorch를 공부하는 중에 쓰이게 되어 새롭게 정리하고자 한다. <br>
저대로 실행하면 의도한 대로 작동하나 doc-string이나 pickle로 직렬화할 때 제대로 작동하지 않는다. <br>

 wraps() is a decorator that is applied to the wrapper function of a decorator. <br>
It updates the wrapper function to look like wrapped function <br>
by copying attributes such as \_\_name\_\_, \_\_doc\_\_ (the docstring), etc. <br>

@functools.wraps(wrapped, assigned=WRAPPER_ASSIGNMENTS, updated=WRAPPER_UPDATES) <br>

- wrapped: wrapper function에 의해 decorated되고자 하는 함수 이름
- assigned: 원래 함수에서 어느 attribute가 직접 짝지어 wrapper function에 들어갈지 튜플로 명시한다. <br>
디폴트로 module, name, qualname, annotations, doc 같은 것들이 들어간다. 거의 건들 이유 없음.
- updated: 어느 wraaper function의 attribute가 원래 함수의 대응되는 attribute가 갱신되는지 튜플로 명시한다. <br>
WRAPPER_UPDATES는 wrapper function의 \_\_dict\_\_를 갱신한다.





reference: https://www.geeksforgeeks.org/python-functools-wraps-function/?ref=gcse

In [5]:
print(sum_func) #sum_func의 이름이 perf_clocked (nested function)은 아니다. 데코레이터 함수가 perf_clocked를 반환하기 때문에 거기에 sum_func를 연결해서 발생한 문제다.
print(help(sum_func))

<function perf_clock.<locals>.perf_clocked at 0x00000285B7C65280>
Help on function perf_clocked in module __main__:

perf_clocked(*args)

None


In [6]:
import pickle

pickle.dumps(sum_func)
# https://hongl.tistory.com/250에서 피클에 대한 언급이 있어서 응용해봤다.

AttributeError: Can't pickle local object 'perf_clock.<locals>.perf_clocked'

In [9]:
from functools import wraps

def perf_clock(func):
    
    # 이렇게 데코레이터를 해 놓으면 모듈 간에 연결되느라 발생한 이름이 엇갈린 것을 해결해준다.
    @wraps(func)
    def perf_clocked(*args):
        # 함수 시작 시간
        st = time.perf_counter()

        # 함수 실행

        result = func(*args)

        # 함수 종료 시간
        et = time.perf_counter() - st
        
        # 실행함수 이름
        name = func.__name__

        # 함수 매개변수
        arg_str = ', '.join(repr(arg) for arg in args)

        # 결과 출력
        print('[%0.5fs] %s(%s) -> %r' % (et, name, arg_str, result))
        return result
    
    return perf_clocked
# enclosing scope end

In [10]:
@perf_clock
def sum_func(*numbers):
    """들어온 인수를 모두 합한다."""
    return sum(numbers)

print(sum_func) 
print(help(sum_func))

pickle.dumps(sum_func)

<function sum_func at 0x00000285B9B88820>
Help on function sum_func in module __main__:

sum_func(*numbers)
    들어온 인수를 모두 합한다.

None


b'\x80\x04\x95\x19\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x08sum_func\x94\x93\x94.'

# GFG examples

단순히 코드가 작동할 것만 생각하면 문제 없이 사용할 수 있다. 그러나 API, library를 작성하면 매우 혼란스러운 상황이 생긴다. <br>
같은 wrpper 함수에 여러 다른 함수를 적용하면 즉, 하나의 decorator에 다른 function을 적용하면 <br>
내부에 무슨 함수가 쓰였는지 name, docstring, help를 써서 알 수가 없다. <br>
이상적으로 wrapping function 대신에 wrapped function의 name, docstring을 보여줘야 한다. <br>


In [16]:
# manual solution은 직접 name과 doc에 직접 값을 할당하는 것이다. 비추천

def a_decorator(func):

    def wrapper(*args, **kwargs):
        """A wrapper function"""
        # Extend some capabilities of func
        func()

    # 일일이 attribute를 연결해줬다.
    wrapper.__name__ = func.__name__
    wrapper.__doc__ = func.__doc__
    return wrapper


@a_decorator
def first_function():
    """This is docstring for the first function"""
    print("first fucntion")

@a_decorator
def second_function(a):
    """This is doctring for the second fucntion"""
    print("second fucntion")


print(first_function.__name__)
print(first_function.__doc__)
print(second_function.__name__)
print(second_function.__doc__)

first_function
This is docstring for the first function
second_function
This is doctring for the second fucntion


In [17]:
print("First Function")
help(first_fucntion)
print("\nSecond Function")
help(second_function)

First Function
Help on function first_fucntion in module __main__:

first_fucntion(*args, **kwargs)
    This is docstring for the first function


Second Function
Help on function second_function in module __main__:

second_function(*args, **kwargs)
    This is doctring for the second fucntion



이렇게 하지 말라는 것에는 2가지 이유가 있다. <br>

1. 함수의 name이 바뀌었지만 그 외의 parameter 같은 다른 요소들의 형태가 바뀌지 않았다. 정확한 정보가 아니다.
2. 모든 함수들을 일일이 이렇게 작성하면 가독성과 생산성이 매우 떨어진다.

따라서 functools.wraps()를 꼭 사용하도록

In [20]:
from functools import wraps
 
def a_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        """A wrapper function"""
 
        # Extend some capabilities of func
        func()
    return wrapper
 
@a_decorator
def first_function():
    """This is docstring for first function"""
    print("first function")
 
@a_decorator
def second_function(a):
    """This is docstring for second function"""
    print("second function")
 
print(first_function.__name__)
print(first_function.__doc__)
print(second_function.__name__)
print(second_function.__doc__)

first_function
This is docstring for first function
second_function
This is docstring for second function


In [ ]:

print("First Function")
help(first_function)
 
print("\nSecond Function")
help(second_function)

First Function
Help on function first_function in module __main__:

first_function()
    This is docstring for first function


Second Function
Help on function second_function in module __main__:

second_function(a)
    This is docstring for second function



# Decorators with parameters in Python



reference: https://www.geeksforgeeks.org/decorators-with-parameters-in-python/?ref=gcse

@decorator(params) <br>
def func_name():

    ''' Function implementation'''

def func_name(): <br>
    ''' Function implementation'''

func_name = (decorator(params))(func_name)


decorator(params)가 호출되면서 함수 객체인 func_obj가 반환되고 <br>
func_obj(func_name)이 호출된다. <br>
inner function 내부에서 연산을 다 수행하고 진짜 function reference는 func_name에 할당된다. <br>
이것이 decorator가 적용된 함수로 기능한다.

In [22]:
params = None

def decorators(*arg, **kwargs):

    def inner(func):
        '''
        Do operations with func
        
        '''
        return func

    return inner # this is the the func_obj mentioned in the above content

@decorators(params)
def func():
    """
        function implementation
    """

In [23]:
# Python code to illustrate
# Decorators basic in Python

# 기본적인 decorator 코드이며 실행 순서를 보여주기 위함이다.
def decorator_func(func):
    print("Inside decorator")

    def inner(*args, **kwars):
        print("Inside inner function")
        print("Decorated the function")
        # do operations with func

        func()

    return inner


@decorator_func
def func_to():
    print("Inside actual function")


func_to()

Inside decorator
Inside inner function
Decorated the function
Inside actual function


In [24]:
# Decorator with parameters in Python

def decorator_func(func):
    print("Inside decorator")

    def inner(*args, **kwargs):
        print("Inside inner function")
        print("Decorated the function")

        func()

    return inner


def func_to():
    print("Inside actual function")

# another way of using decorators
decorator_func(func_to)()

# 함수 형태로 decorator를 표현했다.
# 함수에 함수를 넣고 결과로 나온 함수를 다시 부른다.

Inside decorator
Inside inner function
Decorated the function
Inside actual function


In [26]:
# 데코레이터 형태로 다시 쓴다.

def decorator(*arg, **kwargs):
    print("Inside decorator")

    def inner(func):

        # code functionality here
        print("Inside inner function")
        print("I like", kwargs['like'])

        func()

    # returning inner function
    return inner


@decorator(like = "geeksforgeeks")
def my_func():
    print("Inside actual function")

Inside decorator
Inside inner function
I like geeksforgeeks
Inside actual function


In [27]:

def decorator_func(x, y):

    def Inner(func):

        def wrapper(*args, **kwargs):
            print("I like Geeksforgeeks")
            print("Summation of values - {}".format(x+y))

            func(*args, **kwargs)

        return wrapper
    return Inner


# Not using decorator
def my_func(*args):
    for ele in args:
        print(ele)

# another way of using decorators
decorator_func(12, 15)(my_func)('Geeks', 'for', 'Geeks')


# This example also tells us that outer function parameters can be accessd by the enclosed inner function.

I like Geeksforgeeks
Summation of values - 27
Geeks
for
Geeks


In [29]:
# Multi-level Decorators

def high_decorator(dataType, message1, message2):

    def decorator(func):
        print(message1)

        def wrapper(*args, **kwargs):
            print(message2)
            if all([type(arg) == dataType for arg in args]):
                return func(*args, **kwargs)
            return "Invalid Input"

        return wrapper
    return decorator



@high_decorator(str, "Decorator for 'stringJoin'", "stringJoin started ...")
def stringJoin(*args):
    st = ''
    for i in args:
        st += i
    return st

@high_decorator(int, "Decorator for 'summation'\n", "summation started ...")
def summation(*args):
    summ = 0
    for arg in args:
        summ += arg
    return summ


print(stringJoin("I ", 'like ', 'Geeks', 'for', 'geeks'))
print()
print(summation(19, 2, 8, 533, 67, 981, 119))

Decorator for 'stringJoin'
Decorator for 'summation'

stringJoin started ...
I like Geeksforgeeks

summation started ...
1729


1. Inside the Decorator <br>
![nn](./multi-level_decorator1.png) 

2. Inside the function <br>
![nn](./multi-level_decorator2.png) <br>
Note: Image snapshots are taken using PythonTutor. 


# @decorator VS @decorator(input)
공통점: 둘 다 중첩함수로 받아들인 파라미터를 내부 함수에 적용해서 만든 함수를 반환한다.
 - @decorator: 흔히 볼 수 있는 데코레이터이다. ()가 없다고 해서 인수가 없는 것이 아니다. <br>
 바로 다음 줄에 있는 함수가 디폴트 인수로 쓰인다. <br><br>
 - @decorator(input): 바로 다음 줄에 있는 함수를 인수로 쓰지 않고 다른 또다른 인수를 쓰고자 하는 것이다. <br>
 대신에 내부함수가 바로 다음줄의 함수를 입력으로 받아서 